This is the jupiter notebook that Zhou writes some test functions

In [60]:
import numpy as np
import scipy

In [50]:
import numpy as np

n = 5
Q = np.random.rand(n, n)
Q = np.dot(Q, Q.T)  # Making Q symmetric to ensure it's positive semi-definite
q = np.random.rand(n)
x = np.random.rand(n)

def test_func(x):
    val = 0.5 * np.dot(x.T, np.dot(Q, x)) + np.dot(q.T, x)
    val = val.ravel()[0]
    return val

def test_func_grad(x):
    grad = np.dot(Q, x) + q
    grad = grad.ravel() 
    return grad

def test_func_hess(x):
    return Q


test_func_val = test_func(x)
test_func_grad_val = test_func_grad(x)
test_func_hess_val = test_func_hess(x)

print("test_func_val: \n", test_func_val)
print("test_func_grad_val: \n", test_func_grad_val)
print("test_func_hess_val: \n", test_func_hess_val)



test_func_val: 
 9.789765429386204
test_func_grad_val: 
 [4.81241808 5.54614523 5.14090514 6.7358019  5.26415593]
test_func_hess_val: 
 [[1.6248814  1.40678561 1.37713353 1.52266631 1.363841  ]
 [1.40678561 1.45324591 1.54541516 1.48056116 1.26444254]
 [1.37713353 1.54541516 1.81753313 1.66044483 1.43867821]
 [1.52266631 1.48056116 1.66044483 2.63809107 1.81595291]
 [1.363841   1.26444254 1.43867821 1.81595291 1.50047191]]


The following code is implementation for armijo line search and wolfe line search

In [54]:
def armijo_line_search(f, f_grad, x_curr, tao, c):
    alpha = 1.0  # Starting with a default step size
    p_curr = -f_grad(x_curr)
    while f(x_curr + alpha * p_curr) > f(x_curr) + c * alpha * np.dot(f_grad(x_curr), p_curr):
        alpha *= tao
    return alpha

def wolfe_line_search(f, f_grad, x_curr, p_curr, c1, c2):
    alpha = 1  # Initial step size
    alpha_max = np.inf
    alpha_min = 0
    while True:
        if f(x_curr + alpha * p_curr) > f(x_curr) + c1 * alpha * np.dot(f_grad(x_curr), p_curr):
            alpha_max = alpha
            alpha = 0.5 * (alpha_min + alpha_max)
        elif np.dot(f_grad(x_curr + alpha * p_curr), p_curr) < c2 * np.dot(f_grad(x_curr), p_curr):
            alpha_min = alpha
            if alpha_max == np.inf:
                alpha = 2 * alpha_min
            else:
                alpha = 0.5 * (alpha_min + alpha_max)
        else:
            break
    return alpha


The following code is gradient descent method

In [57]:
def gradient_descent(f, f_grad, x0, method='armijo', tao=0.5, c1=1e-4, c2=0.9, threshold=1e-5, max_iter=1000):
    x_curr = x0

    for _ in range(max_iter):
        grad_curr = f_grad(x_curr)
        if np.linalg.norm(grad_curr) < threshold:
            break
        
        p_curr = -grad_curr
        if method == 'armijo':
            alpha = armijo_line_search(f, f_grad, x_curr, tao, c1)
        elif method == 'wolfe':
            alpha = wolfe_line_search(f, f_grad, x_curr, p_curr, c1, c2)
        else:
            raise ValueError("Invalid method specified. Choose 'armijo' or 'wolfe'.")
        
        x_curr = x_curr + alpha * p_curr

    return x_curr, f(x_curr)
        

In [59]:
x_final, f_final =  gradient_descent(test_func, test_func_grad, x, method='armijo', tao=0.5, c1=1e-4, c2=0.9, threshold=1e-5, max_iter=1000)
print("x_final: \n", x_final)
print("f_final: \n", f_final)

x_final: 
 [ 42.03650355 -70.11482225  49.46740018  12.88251297 -42.25930528]
f_final: 
 -46.47250726512668


The following code is modfied Newton method

In [62]:
def modified_Newton_method(f, f_grad, f_hess, x0, line_search_method='armijo', tau=0.5, c1=1e-4, c2=0.9, threshold=1e-5, max_iter=1000):
    x_curr = np.array(x0, dtype=float)

    for _ in range(max_iter):
        grad_curr = f_grad(x_curr)
        if np.linalg.norm(grad_curr) < threshold:
            break

        # Compute the Hessian and adjust it if necessary
        A = f_hess(x_curr)
        beta = 0.0001
        delta = max(0, -np.min(np.linalg.eigvalsh(A)) + beta)

        while True:
            try:
                # Try Cholesky decomposition to test positive definiteness
                scipy.linalg.cholesky(A + delta * np.eye(len(x_curr)))
                break
            except scipy.linalg.LinAlgError:
                delta = max(2 * delta, beta)
        
        # Adjusted Hessian
        Bk = A + delta * np.eye(len(x_curr))
        # Solve for the search direction
        p_curr = -scipy.linalg.solve(Bk, grad_curr)

        # Perform line search based on the specified method
        if line_search_method == 'armijo':
            alpha = armijo_line_search(f, f_grad, x_curr, tau, c1)
        elif line_search_method == 'wolfe':
            alpha = wolfe_line_search(f, f_grad, x_curr, p_curr, c1, c2)
        else:
            raise ValueError("Invalid line search method specified.")
        
        x_curr = x_curr + alpha * p_curr

        # if current gradient norm is less than threshold, break
        grad_curr = f_grad(x_curr)
        if np.linalg.norm(grad_curr) < threshold:
            break

    return x_curr, f(x_curr)


In [64]:
# running the modified Newton method
x_final, f_final = modified_Newton_method(test_func, test_func_grad, test_func_hess, x, line_search_method='armijo', tau=0.5, c1=1e-4, c2=0.9, threshold=1e-5, max_iter=1000)
print("x_final: \n", x_final)
print("f_final: \n", f_final)

x_final: 
 [  69.65648324 -114.72340133   81.14218871   22.02533092  -71.37345276]
f_final: 
 -54.9071665059195
